# Ejercicio 6
## Detección de Amarillos en tiempo real

### Librerias

In [1]:
import numpy as np
import cv2 as cv

### Limites de Colores

In [2]:
def get_limits(color, delta):
    """
    Obtener los límites de un color en el espacio HSV

    Args:
        color (list): color en BGR
        delta (int): margen de error

    Returns:
        np.array: color en HSV
        np.array: límite inferior
        np.array: límite superior
    """
    # Convertir el color a HSV
    rgb_color = np.uint8([[color]])
    hsv_color = cv.cvtColor(rgb_color, cv.COLOR_BGR2HSV)

    # Definir los limites del color en HSV
    h_value = int(hsv_color[0][0][0])
    min_limit = np.array([max(h_value - delta, 0), 100, 100], dtype=np.uint8)
    max_limit = np.array([min(h_value + delta, 179), 255, 255], dtype=np.uint8)

    return hsv_color[0][0], min_limit, max_limit

In [3]:
# Definir colores en BGR
RED = [0, 0, 255]
GREEN = [0, 255, 0]
BLUE = [255, 0, 0]
YELLOW = [0, 255, 255]

# Obtener límites para cada color
red, red_min, red_max = get_limits(color=RED, delta=15)
green, green_min, green_max = get_limits(color=GREEN, delta=15)
blue, blue_min, blue_max = get_limits(color=BLUE, delta=15)
yellow, yellow_min, yellow_max = get_limits(color=YELLOW, delta=15)

# Mostrar resultados
print(f"Red: {red} -> {red_min} - {red_max}")
print(f"Green: {green} -> {green_min} - {green_max}")
print(f"Blue: {blue} -> {blue_min} - {blue_max}")
print(f"Yellow: {yellow} -> {yellow_min} - {yellow_max}")

Red: [  0 255 255] -> [  0 100 100] - [ 15 255 255]
Green: [ 60 255 255] -> [ 45 100 100] - [ 75 255 255]
Blue: [120 255 255] -> [105 100 100] - [135 255 255]
Yellow: [ 30 255 255] -> [ 15 100 100] - [ 45 255 255]


### Captura de video y deteccion

In [4]:
def gen_contours(image, mask, color_name, color_bgr):
    # Encontrar contornos
    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # Dibujar contornos
    for contour in contours:
        # Filtrar contornos pequeños
        if cv.contourArea(contour) > 300:
            
            # Obtener bounding box
            x, y, w, h = cv.boundingRect(contour)
            # Dibujar bounding box
            cv.rectangle(image, (x, y), (x + w, y + h), color_bgr, 2)
            # Dibujar texto con el color
            cv.putText(image, color_name, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, color_bgr)
    
    return image

In [5]:
# Inicializar la webcam
webcam = cv.VideoCapture(0)

# Inicializar el detector de colores
while True:
    # Leer el frame de la webcam
    ret, video_frame = webcam.read()
    # Verificar si se obtuvo el frame
    if not ret:
        print("ERROR: No se pudo obtener el frame")
        break

    # Convertir el frame a HSV
    hsv_frame = cv.cvtColor(video_frame, cv.COLOR_BGR2HSV)

    # Crear una mascara para cada color
    red_mask = cv.inRange(hsv_frame, red_min, red_max)
    green_mask = cv.inRange(hsv_frame, green_min, green_max)
    blue_mask = cv.inRange(hsv_frame, blue_min, blue_max)
    yellow_mask = cv.inRange(hsv_frame, yellow_min, yellow_max)

    # Aplicar una transformación morfológica a cada máscara, 
    # Dilation para unir regiones cercanas
    # y bitwise_and para obtener la región de interés
    # Kernel
    kernel = np.ones((5, 5), np.uint8)  # Matriz de 5x5 con unos

    # Rojo
    red_mask = cv.dilate(red_mask, kernel)
    res_red = cv.bitwise_and(video_frame, video_frame, mask=red_mask)

    # Verde
    green_mask = cv.dilate(green_mask, kernel)
    res_green = cv.bitwise_and(video_frame, video_frame, mask=green_mask)

    # Azul
    blue_mask = cv.dilate(blue_mask, kernel)
    res_blue = cv.bitwise_and(video_frame, video_frame, mask=blue_mask)

    # Amarillo
    yellow_mask = cv.dilate(yellow_mask, kernel)
    res_yellow = cv.bitwise_and(video_frame, video_frame, mask=yellow_mask)

    # Crear una ventana para mostrar el frame y el contorno
    video_frame = gen_contours(video_frame, red_mask, "Red", RED)
    video_frame = gen_contours(video_frame, green_mask, "Green", GREEN)
    video_frame = gen_contours(video_frame, blue_mask, "Blue", BLUE)
    video_frame = gen_contours(video_frame, yellow_mask, "Yellow", YELLOW)

    # Mostrar el frame
    cv.imshow("Frame", video_frame)

    # Salir si se presiona la tecla 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        webcam.release()    # Liberar la webcam
        cv.destroyAllWindows()  # Cerrar las ventanas
        break

ERROR: No se pudo obtener el frame
